<a href="https://colab.research.google.com/github/dcgmechanics/GenieVA/blob/master/T2GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=7><center>Torrent to Google Drive</center></font>

<font size=4><center>This notebook is a stripped down version of <a href="https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb" target="_blank">MiXLab</a>.</center></font>

# <font size=6><center>✦ *Colab Stay Alive* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Colab Stay Alive</font>
# @markdown This cell runs a JS code that will automatically press the reconnect button when you got disconnected due to idle.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

If the cell above doesn't work, try to run one of these codes below on your browser's developer tool/console.

</br>

>Code 1</br>(credit to rockyourcode)
<pre><code>function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}

setInterval(ClickConnect, 60000)</code></pre>

</br>

> Code 2</br>(credit to Kavyajeet Bora on stack overflow)
<pre><code>function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)</code></pre>

</br>

> Code 3
<pre><code>function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)</code></pre>

</br>

> Code 4</br>(credit to Stephane Belemkoabga on stack overflow)
<pre><code>function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)</code></pre>

# <font size=6><center>✦ *Mount Google Drive* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount/Unmount Google Drive</font>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
# @markdown <p>Google Drive will be mounted to:</p>
# @markdown > "/content/drive/MyDrive" for the standard drive</br>"/content/drive/Shareddrives" for the team/shared drives
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import drive

drive.mount._DEBUG = False

if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Force re-mount Google Drive</font>
# ================================================================ #

import IPython
from IPython.display import HTML, clear_output

drive.mount("/content/drive", force_remount=True)

clear_output()

# <font size=6><center>✦ *libtorrent* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Install libtorrent</font>
# ================================================================ #

from IPython.display import clear_output

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Add Torrent from File</font>
# @markdown <p><font size=5><center>How to change the download location:</center></font></br><font size=4>1. Double click the cell to show its code</br>2. Find this line: "save_path": "/content/downloads",</br>3. Change /content/downloads to your path</font></p>
# @markdown > You can run this cell as many time as you want.
# ================================================================ #

import os
from IPython.display import clear_output
from google.colab import files

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

source = files.upload()
params = {
    "save_path": "/content/downloads",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Add Torrent from Magnet Link</font>
# @markdown <p><font size=5><center>How to change the download location:</center></font></br><font size=4>1. Double click the cell to show its code</br>2. Find this line: params = {"save_path": "/content/downloads"}</br>3. Change /content/downloads to your path</font></p>
# @markdown > You can run this cell as many time as you want.
# ================================================================ #

import os
from IPython.display import clear_output

if os.path.exists("/content/downloads"):
  pass
else:
  os.mkdir("/content/downloads")

params = {"save_path": "/content/downloads"}

while True:
    magnet_link = input("Paste the magnet link here or type exit to stop:\n")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] libtorrent</font>
# ================================================================ #

import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


# <font size=6><center>✦ *SimpleTorrent* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] SimpleTorrent</font>
# ================================================================ #

Install_old_version = False
Auto_UP_Gdrive = False
AUTO_MOVE_PATH = "/content/drive/MyDrive"
force_change_version = "" # https://github.com/boypt/simple-torrent/releases
rclone_DestinationPath = ""
USE_FREE_TOKEN = True
TOKEN = ""
PORT_FORWARD = "ngrok"

import os
import time
import pathlib 
import urllib.request, requests, tarfile
from subprocess import Popen
from IPython.display import clear_output
 
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
 
if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    findProcess,
    loadingAn,
    displayUrl,
    PortForward_wrapper
)
# making enviroment for simple-torrent
os.makedirs('downloads', exist_ok=True)
os.makedirs('torrents', exist_ok=True)
os.makedirs('tools/simple-torrent', exist_ok=True)
 
def generateCmd(src, dst):
  FAST_LIST = True
  PATH_RClone_Config = "/usr/local/sessionSettings"
  cmd = f'rclone move "{src}" "{dst}" ' \
    f'--config {PATH_RClone_Config}/rclone.conf ' \
    f'{"--fast-list" if FAST_LIST else ""} --user-agent "Mozilla" ' \
    '--transfers 20 --checkers 20 --drive-server-side-across-configs ' \
    '-c --buffer-size 256M --drive-chunk-size 256M ' \
    '--drive-upload-cutoff 256M --drive-acknowledge-abuse ' \
    '--drive-keep-revision-forever --tpslimit 95 --tpslimit-burst 40 ' \
    '--stats-one-line --stats=5s -v'
  return cmd
 
 
if Auto_UP_Gdrive:
  data = """#!/bin/bash
 
dir=${CLD_DIR}
path=${CLD_PATH}
abp="${dir}/${path}"
type=${CLD_TYPE}
 
if [[ ${type} == "torrent" ]]; then
"""
  nUpload = """    
    
    #Upload to Gdrive
    #mkdir -p "%s/$(dirname "${path}")"
    mv "${abp}" "%s/${path}"
 
    """ % (AUTO_MOVE_PATH, AUTO_MOVE_PATH)
  
  rcloneUpload = """
    #You can also use rcone move file to remote
    %s
    """ % generateCmd(r"${abp}", rclone_DestinationPath)
  
  end = """
fi
"""
  data = data + (rcloneUpload if rclone_DestinationPath else nUpload) + end
  with open(pathDoneCMD, 'w') as w:
    w.write(data)
  os.chmod(pathDoneCMD, 0o755)
else:
  try:
    os.unlink(pathDoneCMD)
  except FileNotFoundError:
    pass
  
configPath = pathlib.Path('tools/simple-torrent/cloud-torrent.json')
configsdata = r"""
{{
  "AutoStart": true,
  "EngineDebug": false,
  "MuteEngineLog": true,
  "ObfsPreferred": true,
  "ObfsRequirePreferred": false,
  "DisableTrackers": false,
  "DisableIPv6": false,
  "DownloadDirectory": "/content/downloads/",
  "WatchDirectory": "torrents/",
  "EnableUpload": true,
  "EnableSeeding": false,
  "IncomingPort": 50007,
  "DoneCmd": "{}/doneCMD.sh",
  "SeedRatio": 1.5,
  "UploadRate": "High",
  "DownloadRate": "Unlimited",
  "TrackerListURL": "https://trackerslist.com/best.txt",
  "AlwaysAddTrackers": true,
  "ProxyURL": ""
}}
""".format(HOME)
with open(configPath, "w+") as configFile:
  configFile.write(configsdata)
#####################################
 
# Simple Torrent installing ...
loadingAn()
if not os.path.isfile("tools/simple-torrent/cloud-torrent"):
    filename = 'tools/simple-torrent/cloud-torrent_linux_amd64.gz'
    if Install_old_version:
      latestTag = '1.2.3'
    else:
      latestTag = requests.get(
        "https://api.github.com/repos/boypt/simple-torrent/releases/latest"
        ).json()['tag_name']
    url = "https://github.com/boypt/simple-torrent/releases/download/" \
      f"{latestTag}/{filename[21:]}"
 
    urllib.request.urlretrieve(url, filename)
    import gzip, shutil
    with gzip.open(filename,
                   'rb') as f_in:
        with open('tools/simple-torrent/cloud-torrent', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.chmod('tools/simple-torrent/cloud-torrent', 0o775)
    os.remove(filename)
    
#Opening cloud-torrent in background
if not findProcess("cloud-torrent", "SimpleTorrent"):
  PORT = 4444
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    cmdC = f'./cloud-torrent --port {PORT} ' \
        '-t Simple-Torrent ' \
        '-c cloud-torrent.json ' \
        '--host 0.0.0.0'
    for run in range(10):    
      Popen(cmdC.split(), cwd='tools/simple-torrent')
      time.sleep(3)
      try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
        break
      except:
        print("Error: Simple-Torrent not starting. Retrying ...")
 
# START_SERVER
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['simple-torrent', 4444, 'http'], 
    ['peerflix-server', 4445, 'http']], 'us', 
    [f"{HOME}/.ngrok2/sTpS.yml", 4040]
).start('simple-torrent')
displayUrl(Server, pNamU='SimpleTorrent : ')

# <font size=6 color=red><center>✦ *TESTING AREA* ✦</center></font>

## <font size=6><center>✦ *qBittorrent* ✦</center></font>

### <font size=6><center>✦ *qBittorrent 1* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Install qBittorrent</font>
# ================================================================ #

import IPython
from IPython.display import HTML, clear_output

!apt install qbittorrent-nox
clear_output()
!qbittorrent-nox

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← ngrok</font>
ngrok_Authtoken = "" #@param {type:"string"}
# @markdown > Click <a href="https://dashboard.ngrok.com/signup" target="_blank">here</a> to register for a free ngrok account.</br>Click <a href="https://dashboard.ngrok.com/auth/your-authtoken" target="_blank">here</a> to copy your ngrok Authtoken.
# ================================================================ #

import os, IPython
from zipfile import ZipFile
from urllib.request import urlretrieve
from IPython.display import HTML, clear_output

def install_ngrok():
  if os.path.exists('/usr/local/bin/ngrok'):
    return
  else:
    url = 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
    urlretrieve(url, 'ngrok-amd64.zip')
      
    with ZipFile('ngrok-amd64.zip', 'r') as zip_ref:
      zip_ref.extractall('/usr/local/bin/')
    os.chmod('/usr/local/bin/ngrok', 0o755)
    os.unlink('ngrok-amd64.zip')

    if os.path.exists('/content/ngrok-amd64.zip'):
      os.remove('/content/ngrok-amd64.zip')
    else:
      return

install_ngrok()

if ngrok_Authtoken != "":
  !ngrok authtoken "$ngrok_Authtoken"
  # clear_output()
else:
  print("The ngrok_Authtoken field is empty!")

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] qBittorent</font>
# ================================================================ #

import threading, time, requests, json, subprocess

def torrent(port):
  command = subprocess.Popen(['qbittorrent-nox', f'--webui-port={port}'])

def ngrok(port):
  ngrok_cmd = subprocess.Popen(['ngrok', 'http', str(port)])    
  localhost_url = "http://localhost:4040/api/tunnels"

  time.sleep(1)
  tunnel_url = requests.get(localhost_url).text
  json_data = json.loads(tunnel_url)

  tunnel_url = json_data['tunnels'][0]['public_url']
  tunnel_url = tunnel_url.replace("https", "http")
  print('Running at localhost: ' + str(port))
  print(tunnel_url)

if __name__ == '__main__':
  port = 9999

  thread_torrent = threading.Thread(target = torrent, args=(int(port),))
  thread_ngrok = threading.Thread(target = ngrok, args=(int(port),))

  thread_torrent.start()
  print('Torrent server started!\n')

  time.sleep(5)
  print('Establishing secure connection!')

  thread_ngrok.start()
  print('Secure connection established...\n')
  print('====================')
  print('Username: admin')
  print('password: adminadmin')
  print('====================\n')

  thread_ngrok.join()
  thread_torrent.join()

### <font size=6><center>✦ *qBittorrent 2* ✦</center></font>

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] qBittorrent</font>
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost"]
# @markdown > -- qBittorrent Default Login --</br>Username: admin</br>Password: adminadmin
# ================================================================ #

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)

def addUtils():
    if not checkAvailable("/usr/local/sessionSettings"):
        runSh("mkdir -p -m 777 /usr/local/sessionSettings")
    if not checkAvailable("/content/upload.txt"):
        runSh("touch /content/upload.txt")
    if not checkAvailable("checkAptUpdate.txt", userPath=True):
        runSh("apt update -qq -y")
        runSh("apt-get install -y iputils-ping")

def configTimezone(auto=True):
    if checkAvailable("timezone.txt", userPath=True):
        return
    if not auto:
        runSh("sudo dpkg-reconfigure tzdata")
    else:
        runSh("sudo ln -fs /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime")
        runSh("sudo dpkg-reconfigure -f noninteractive tzdata")

def uploadQBittorrentConfig():
    if checkAvailable("updatedQBSettings.txt", userPath=True):
        return
    runSh(
        "mkdir -p -m 666 /content/qBittorrent /root/.qBittorrent_temp /root/.config/qBittorrent"
    )
    runSh(
        "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/configurations/qbittorrent/qBittorrent.conf \
            -O /root/.config/qBittorrent/qBittorrent.conf"
    )

def prepareSession():
    if checkAvailable("ready.txt", userPath=True):
        return
    else:
        addUtils()
        configTimezone()
        uploadQBittorrentConfig()

def installQBittorrent():
    if checkAvailable("/usr/bin/qbittorrent-nox"):
        return
    else:
        try:
            runSh("apt-get install qbittorrent-nox -qq -y")
        except:
            raise Exception('Failed to install qBittorrent!')

def startQBService():
    prepareSession()
    installQBittorrent()
    if not findProcess("qbittorrent-nox", "-d --webui-port"):
        runSh(f"qbittorrent-nox -d --webui-port={QB_Port}")
        time.sleep(1)

QB_Port = 10001
loadingAn()
startQBService()

clear_output()

server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['qbittorrent', QB_Port, 'http']], 'sa', [f"{HOME}/.ngrok2/qbittorrent.yml", 4088]).start('qbittorrent', displayB=False)
displayUrl(server, pNamU='qBittorrent : ')